In [1]:
import pandas as pd
from datetime import datetime, timezone, timedelta


In [53]:
from netCDF4 import Dataset
import numpy as np 
import os
import plotly.graph_objects as go
from plotly.offline import plot, iplot
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
import numpy as np

config = dict({
                'displayModeBar': True,
                'responsive': False,
                'staticPlot': False,
                'displaylogo': False,
                'showTips': False,
                })

# Simplify Plotting Schemes:
col1 = px.colors.qualitative.Plotly[0]
col2 = px.colors.qualitative.Plotly[1]
col3 = px.colors.qualitative.Plotly[2]
col4 = px.colors.qualitative.Plotly[3]
col5 = px.colors.qualitative.Plotly[4]
col6 = px.colors.qualitative.Plotly[5]



def read_nc_file( filename, variables):
    ''' This function reads the TIEGCM .nc files and saves the given input variables to a dictionary.
        The breakloop feature is here so that if the file doesn't exist the code can still continue.  '''
    status = os.path.exists(filename)
    
    if status == True:
        data = {}
        for i, var_names in enumerate(variables):
            ncid =  Dataset(filename,"r+", format="NETCDF4")# filename must be a string
            varData = ncid.variables
            data[var_names] = np.array(varData[var_names])  
    elif status == False:
        print('No File Found', filename )
        breakloop = True
        data = 0
        return( data , breakloop)
    breakloop = False
    return(data,breakloop )


In [62]:
path_to_f107 = '/data/data_geodyn/gpi_1960001-2021243_f107aDaily.nc'
variables = ['year_day', 'f107d', 'f107a', 'kp']
f107_data = read_nc_file(path_to_f107, variables)

date = []
kp_list = []
f107d_list = []
f107a_list  = []

for i,val in enumerate([
                        2018292,
                        2018293,
                        2018294,
                        2018295,
                        2018296,                        
                        2018297,
                        2018298,
                        2018299,
                        #     
                        2018304, 
                        2018305, 
                        2018306, 
                        2018307, 
                        2018308, 
                        #     
                        2018313, 
                        2018314, 
                        2018315, 
                        2018316, 
                        2018317, 
                        2018318,
                        2018319, 
                        2018320, 
                        2018321, 
                        2018322, 
                        2018323, 
                        2018324,
                        2018325, 
                        2018326, 
                        2018327,
                        2018328,
                        #
                        2018335,
                        2018336,
                        2018337,
                        #
                        2018349, 
                        2018350, 
                        2018351,
                        2018352,
                        #
                        2018356,
                        2018357,
                        2018358,
                        #
                        2018365,
                        2019001,
                        2019002,
                        2019003,
                        2019004,
                        2019005,
                        2019006,
                        2019007,
                        2019008,
                       ]):
    
    index = f107_data[0]['year_day']==val
    kp_list.append(f107_data[0]['kp'][index][0])
    f107d_list.append(f107_data[0]['f107d'][index][0])
    f107a_list.append(f107_data[0]['f107a'][index][0])
    
    date.append(pd.to_datetime( str(val), format='%Y%j'))
kp_expand = []
for i in kp_list:
    for ii in i:
        kp_expand.append(ii)
        
        
        



fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=(['F10.7', 'Kp']),
    vertical_spacing = 0.2)





fig.add_trace(go.Scattergl(  x=date,
                             y=f107d_list,
                             name= 'f107d',
                             mode='markers',
                             opacity=1,
                             marker=dict(
                                color=col2, 
                                size=8,
                                        ),
                             showlegend=True,
                          ),
                              secondary_y=False,
                               row=1, col=1,
                          )

fig.add_trace(go.Scattergl(  x=date,
                             y=f107a_list,
                             name= 'f107a',
                             mode='markers',
                             opacity=1,
                             marker=dict(
                                color=col1, 
                                size=8,
                                        ),
                             showlegend=True,
                          ),
                              secondary_y=False,
                               row=1, col=1,
                          )


fig.add_trace(go.Scattergl(  x=np.arange(1,np.size(kp_expand)),
                             y=kp_expand,
                             name= 'kp',
                             mode='markers',
                             opacity=1,
                             marker=dict(
                                color=col3, 
                                size=8,
                                        ),
                             showlegend=True,
                          ),
                              secondary_y=False,
                               row=2, col=1,
                          )

fig.update_layout(title= 'NGDC Geophysical Indices -- F10.7 and Kp',
                autosize=True,
                font=dict(size=14),
                legend= {'itemsizing': 'constant'})

fig.update_yaxes( title="sfu", row=1, col=1)
fig.update_yaxes( title="Kp", row=2, col=1)

fig.show(config=config)


2018287

### TIEGCM manual functions

In [2]:
from netCDF4 import Dataset
import numpy as np 
import os
import warnings
warnings.filterwarnings("ignore")

def read_tiegcm( filename, variables):
    ''' This function reads the TIEGCM .nc files and saves the given input variables to a dictionary.
        The breakloop feature is here so that if the file doesn't exist the code can still continue if placed in a loop.'''
    status = os.path.exists(filename)
 
    if status == True:
        data = {}
        for i, var_names in enumerate(variables):
            ncid =  Dataset(filename,"r+", format="NETCDF4")# filename must be a string
            varData = ncid.variables
            data[var_names] = np.array(varData[var_names])  
    elif status == False:
        print('No File Found', filename )
        breakloop = True
        data = 0
        return( data , breakloop)
    breakloop = False
    return(data,breakloop )


def find_modelgrid_base_index(TIEGCM, lon_sat, lat_sat, time_sat):
    """ 
    Interpolation function: (3 / 5)

    This function locates the grid points that make up thr square surrounding out desired point.
    """

    lon_start = -180 # first longitude point of the model grid (deg)
    dlon = 5  # difference between adjacent longitude grid points (deg)
    index_lon0 = int(np.mod(np.floor( (lon_sat - lon_start )/dlon ), 72 ) ) 


    lat_start = -87.5 # first longitude point of the model grid (deg)
    dlat = 5 # difference between adjacent longitude grid points (deg)
    index_lat0 = int(np.mod(np.floor( (lat_sat - lat_start)/dlat ), 36 ) )

    time_start = 0 # first longitude point of the model grid (deg)
    dtime = 1 # difference between adjacent longitude grid points (deg)
    index_time = int((np.mod(np.floor( (time_sat - time_start)/dtime ), 24 ) )) 

    if index_lon0 == 71:
        index_lon1 = 0
    else:
        index_lon1 = index_lon0 + 1


    if index_lat0 >= 35 :
        index_lat1 = 35
        NPole_Flag = True
        SPole_Flag = False 

    elif index_lat0 <= 0: 
        index_lat1 = 0
        SPole_Flag = True
        NPole_Flag = False
    else: 
        index_lat1 = index_lat0 + 1
        NPole_Flag = False 
        SPole_Flag = False 


    return(index_lon0, index_lat0, index_lon1, index_lat1, index_time, NPole_Flag, SPole_Flag )


def DEN_and_ZG_vertprofs(TIEGCM, lon_sat, lat_sat, time_sat, param):
    '''
    Interpolation function: (2 / 5)

    This function finds the vertical profiles of the four grid points around a desired point.
    '''
    
    #### First we find the four grid points:
    (indexlon0, 
    indexlat0, 
    indexlon1, 
    indexlat1, 
    indexut, 
    NPole_Flag, 
    SPole_Flag) = find_modelgrid_base_index(TIEGCM, lon_sat, lat_sat, time_sat)
    grid_vals = pd.DataFrame(data = {'lon0' : TIEGCM['lon'][indexlon0] ,\
                                 'lonindex0': indexlon0 ,\
                                 'lon1'     :TIEGCM['lon'][indexlon1] ,\
                                 'lonindex1':indexlon1 ,\
                                 'lat0'     :TIEGCM['lat'][indexlat0] ,\
                                 'latindex0':indexlat0 ,\
                                 'lat1'     :TIEGCM['lat'][indexlat1] ,\
                                 'latindex1':indexlat1  }, index=[0])
    

#     print('UT index',indexut)
#     print('UT value',TIEGCM['ut'][indexut])
#     print()
#     print('Lon0 index',indexlon0)
#     print('Lon0 value',TIEGCM['lon'][indexlon0])
#     print('Lon1 index',indexlon1)
#     print('Lon1 value',TIEGCM['lon'][indexlon1])
#     print()
#     print('Lat0 index',indexlat0)
#     print('Lat0 value',TIEGCM['lat'][indexlat0])
#     print('Lat1 index',indexlat1)
#     print('Lat1 value',TIEGCM['lat'][indexlat1])
    
#     print('alt index',indexut)
#     print('alt value',TIEGCM['ut'][indexut])

    DEN_prof00 = TIEGCM[param][indexut, :-1, indexlat0, indexlon0]
    DEN_prof10 = TIEGCM[param][indexut, :-1, indexlat1, indexlon0]
    DEN_prof01 = TIEGCM[param][indexut, :-1, indexlat0, indexlon1]
    DEN_prof11 = TIEGCM[param][indexut, :-1, indexlat1, indexlon1]
    

    ZG_prof00 = TIEGCM['ZG'][indexut, :-1, indexlat0, indexlon0]
    ZG_prof10 = TIEGCM['ZG'][indexut, :-1, indexlat1, indexlon0]
    ZG_prof01 = TIEGCM['ZG'][indexut, :-1, indexlat0, indexlon1]
    ZG_prof11 = TIEGCM['ZG'][indexut, :-1, indexlat1, indexlon1]

    DEN_df = pd.DataFrame(data = {'00' :DEN_prof00 ,\
                                  '10' :DEN_prof10 ,\
                                  '01' :DEN_prof01 ,\
                                  '11' :DEN_prof11 ,\
                                })
    ZG_df = pd.DataFrame(data = {'00' :ZG_prof00 ,\
                                  '10' :ZG_prof10 ,\
                                  '01' :ZG_prof01 ,\
                                  '11' :ZG_prof11 ,\
                                })
    return(DEN_df, ZG_df, grid_vals)



def interp_DENSITY_to_altitude(DEN_df, ZG_df, height_sat):
    '''
    Interpolation function: (4 / 5)

    This function interpolates Density in the log space then converts it back'''
    density_surf_at_height_sat = np.zeros(np.size(ZG_df.columns))
    i = 0
    for col in ZG_df.columns:
        xp = ZG_df[col][:].values
        fp = DEN_df[col][:].values
        xval = height_sat
        density_surf_at_height_sat[i] =  np.exp( np.interp(xval, xp, np.log(fp))  )
        i += 1
    return(density_surf_at_height_sat)



def interp_param_to_altitude(df, ZG_df, height_sat):
    
    '''
    Interpolation function: (5 / 5)

    This function interpolates all other parameters (not density) using linear interpolation
    '''

    param_surf_at_height_sat = np.zeros(np.size(ZG_df.columns))
    i = 0
    for col in ZG_df.columns:
        xp = ZG_df[col][:].values
        fp = df[col][:].values
        xval = height_sat
        param_surf_at_height_sat[i] =  np.interp(xval, xp, fp)  
        i += 1
    return(param_surf_at_height_sat)





def interpolate_tiegcm(TIEGCM, lon_sat, lat_sat, time_sat, height_sat, param):
    '''
    Interpolation function: (1 / 5)
    
    This function finds the interpolated value of a TIEGCM parameter.  
    Interpolated to a desired lon,lat,alt
    '''
    
    # Find the vertical profiles of the four grid points making up the grid square around your desired point.
#     print('time_sat:', time_sat)
    df, ZG_df, grid_vals = DEN_and_ZG_vertprofs(TIEGCM, lon_sat, lat_sat, time_sat, param)
    
    if param == 'DEN':
        param_surf_at_height_sat = interp_DENSITY_to_altitude(df, ZG_df, height_sat)
    else:
        param_surf_at_height_sat = interp_param_to_altitude(df, ZG_df, height_sat)


    xval = lon_sat

    x_lon = [grid_vals['lon0'][0], grid_vals['lon1'][0]]
    f_den0 = [param_surf_at_height_sat[0], param_surf_at_height_sat[1]]
    interp_lon0 =   np.interp(xval, x_lon, f_den0  )

    f_den1 = [param_surf_at_height_sat[2], param_surf_at_height_sat[3]]
    interp_lon1 =   np.interp(xval, x_lon, f_den1  )

    final_interped_value = interp_lon0 + ((lat_sat - grid_vals['lat0'][0]) / (grid_vals['lat1'][0] - grid_vals['lat0'][0]))*(interp_lon1 - interp_lon0)

    return(final_interped_value)


# def gravity(h):
#     """For h in same units as R_e  (computes in cm as artifact of old code)"""
#     return g_0*pow(R_e/(R_e + h), 2)



# def interp_to_altitude(DEN_df, ZG_df, height_sat):
#     density_surf_at_height_sat = np.zeros(np.size(ZG_df.columns))
#     i = 0
#     for col in ZG_df.columns:
#         xp = ZG_df[col][:].values*1e-5
#         fp = DEN_df[col][:].values
#         xval = height_sat
#         density_surf_at_height_sat[i] =  np.exp( np.interp(xval, xp, np.log(fp))  )
#         i += 1
#     return(density_surf_at_height_sat)




# def interp_lon_lat(TIEGCM, lon_sat, lat_sat, time_sat, height_sat, param):
#     DEN_df, ZG_df, grid_vals = DEN_and_ZG_vertprofs(TIEGCM, lon_sat, lat_sat, time_sat, param)
#     density_surf_at_height_sat = interp_to_altitude(DEN_df, ZG_df, height_sat)

#     xval = lon_sat

#     x_lon = [grid_vals['lon0'][0], grid_vals['lon1'][0]]
#     f_den0 = [density_surf_at_height_sat[0], density_surf_at_height_sat[1]]
#     interp_lon0 =   np.interp(xval, x_lon, f_den0  )

    
#     f_den1 = [density_surf_at_height_sat[2], density_surf_at_height_sat[3]]
#     interp_lon1 =   np.interp(xval, x_lon, f_den1  )
    
#     density = interp_lon0 + ((lat_sat - grid_vals['lat0'][0]) / (grid_vals['lat1'][0] - grid_vals['lat0'][0]))*(interp_lon1 - interp_lon0)
# #     print(density)
#     return(density)

### Load the Data

In [3]:
# %load_ext autoreload
# %autoreload 2

# import sys  
# import pickle 
# sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
# from PYGEODYN import Pygeodyn


# Obj_Geodyn = {}
# for i,val in enumerate(['msis2','tiegcm_oc']):
#     run_settings = '/data/geodyn_proj/validation/NoCD_Arc24_AllDensModels_settings/NoCdArc24_'+val+'.yaml'

#     ### Load the data into an object
#     Obj_Geodyn[val] = Pygeodyn(run_settings)
#     Obj_Geodyn[val].getData()
# #     Obj_Geodyn.post_getData_ValidationMetrics()

In [4]:
arc = '2018.313'



In [5]:
# Obj_Geodyn['msis2'].__dict__['Density'][arc]


In [6]:
# den_df = Obj_Geodyn['tiegcm_oc'].__dict__['Density'][arc]


# vals  = np.arange(den_df.index[0],den_df.index[-1]+1)
# df = den_df.set_index('Date',drop=False ) 
# df['i_vals'] = vals
# index_date = df.loc[df.index.max()]['i_vals'].min()

# den_df = den_df[:index_date-4000]

In [7]:
# den_df

In [8]:
# den_df['LON'] = (den_df['Lon'] - 180) 
# den_df['LAT'] = den_df['Lat']

In [9]:
# den_df

### Load the MSIS2 inputs

In [10]:
path_msisin = '/data/data_geodyn/results/icesat2/msis2/msis2_acceloff_valid_ctrlrun/DENSITY/icesat2_2018313_24hr.msis2.NoCD_Arc24_msisin'

msisinputs = pd.read_csv(path_msisin, 
                    dtype=object,
                    names = ['YYMMDD',
                             'HHMMSS',
                             'Height_kilometers',
                             'Lat',
                             'Lon',
                                 ],
                    sep = '\s+',
                    )

sat_time1 = list(msisinputs['YYMMDD'])  #"031115" #  
sat_time2 = list(msisinputs['HHMMSS'])  #"120000" #1068897600        
sattime   =    [x+y   for x,y   in zip(sat_time1, sat_time2)]
sattime   =    [datetime.strptime(x, '%y%m%d%H%M%S')   for x   in sattime ]
sattime   =    [datetime.timestamp(x)   for x   in sattime ]

msisinputs['sattime_utctimestamp'] = sattime
msisinputs['Lon'] = msisinputs['Lon'].astype(float)
msisinputs['Lat'] = msisinputs['Lat'].astype(float)



timeHHMMSS = [] 
for i,val in enumerate(msisinputs['HHMMSS'].values.astype(int)):
    # print(len(str(val)))
    if len(str(val)) == 1:
        timehhmmss_val = '00000'+ str(val)
        timeHHMMSS.append(timehhmmss_val)
    elif len(str(val)) == 2:
        timehhmmss_val = '0000'+ str(val)
        timeHHMMSS.append(timehhmmss_val)
    elif len(str(val)) == 3:
        timehhmmss_val = '000'+ str(val)
        timeHHMMSS.append(timehhmmss_val)
    elif len(str(val)) == 4:
        timehhmmss_val = '00'+ str(val)
        timeHHMMSS.append(timehhmmss_val)
    elif len(str(val)) == 5:
        timehhmmss_val = '0'+ str(val)
        timeHHMMSS.append(timehhmmss_val)
    else:
        timeHHMMSS.append(str(val))
msisinputs['timeHHMMSS'] = timeHHMMSS
YR = int(18)

YYMMDD_list = msisinputs['YYMMDD'].astype(int).astype(str)
timeHHMMSS_list = msisinputs['timeHHMMSS'].astype(str)

if YR < 10:
    year    = ['200' + x[:1]  for x in YYMMDD_list]
    month   = [        x[1:3] for x in YYMMDD_list]
    day     = [        x[3:]  for x in YYMMDD_list]
    hours   = [        x[:2]  for x in timeHHMMSS_list]
    minutes = [        x[2:4] for x in timeHHMMSS_list]
    secs    = [        x[4:]  for x in timeHHMMSS_list]
else:
    year    = ['20' + x[:2]  for x in YYMMDD_list]
    month   = [       x[2:4] for x in YYMMDD_list]
    day     = [       x[4:]  for x in YYMMDD_list]
    hours   = [       x[:2]  for x in timeHHMMSS_list]
    minutes = [       x[2:4] for x in timeHHMMSS_list]
    secs    = [       x[4:]  for x in timeHHMMSS_list]
#--------------------------------------------------------
msisinputs['year']  = year
msisinputs['month'] = month
msisinputs['day']   = day
msisinputs['hours']  = hours
msisinputs['minutes'] = minutes
msisinputs['secs']  = secs
#--------------------------------------------------------
year   = list(map(int, msisinputs['year'].values))
month  = list(map(int, msisinputs['month'].values))
day    = list(map(int, msisinputs['day'].values))
hour   = list(map(int, msisinputs['hours'].values))
minute = list(map(int, msisinputs['minutes'].values))
second = list(map(int, msisinputs['secs'].values))

DATE = list(map(datetime, year,month, day, hour,minute,second ))
msisinputs.insert(0, 'Date', DATE)

del msisinputs['timeHHMMSS']
del msisinputs['year']
del msisinputs['month']
del msisinputs['day']
del msisinputs['hours']
del msisinputs['minutes']
del msisinputs['secs']

In [11]:
path_msisingpi = '/data/data_geodyn/results/icesat2/msis2/msis2_acceloff_valid_ctrlrun/DENSITY/icesat2_2018313_24hr.msis2.NoCD_Arc24_msisin_gpiflux'

msisinput_gpi = pd.read_csv(path_msisingpi, 
                names = [
                        'YYMMDD',
                        'HHMMSS',
                        'flux_daily',
                        'flux_avg',
                        'Ap1',
                        'Ap2',
                        'Ap3',
                        'Ap4',
                        'Ap5',
                        'Ap6',
                        'Ap7',
                        'Kp',
                      ],
                sep = '\s+',
                )


In [12]:
msisinputs

,Date,YYMMDD,HHMMSS,Height_kilometers,Lat,Lon,sattime_utctimestamp
0,2018-11-09 00:01:09,181109,000109,482.623,11.309,-145.122,1.541722e+09
1,2018-11-09 00:01:19,181109,000119,482.599,11.949,-145.187,1.541722e+09
2,2018-11-09 00:01:29,181109,000129,482.580,12.589,-145.252,1.541722e+09
3,2018-11-09 00:01:39,181109,000139,482.564,13.229,-145.317,1.541722e+09
4,2018-11-09 00:01:49,181109,000149,482.553,13.869,-145.382,1.541722e+09
...,...,...,...,...,...,...,...
59943,2018-11-09 23:40:49,181109,234049,483.779,31.720,-141.577,1.541807e+09
59944,2018-11-09 23:40:59,181109,234059,483.872,32.359,-141.650,1.541807e+09
59945,2018-11-09 23:41:09,181109,234109,483.968,32.997,-141.723,1.541807e+09
59946,2018-11-09 23:41:19,181109,234119,484.067,33.636,-141.796,1.541807e+09


In [13]:
msisinput_gpi

,YYMMDD,HHMMSS,flux_daily,flux_avg,Ap1,Ap2,Ap3,Ap4,Ap5,Ap6,Ap7,Kp
0,181109,109,70.37,68.91,0.0,5.0,5.0,6.0,4.0,11.38,8.25,1.33
1,181109,119,70.37,68.91,0.0,5.0,5.0,6.0,4.0,11.38,8.25,1.33
2,181109,129,70.37,68.91,0.0,5.0,5.0,6.0,4.0,11.38,8.25,1.33
3,181109,139,70.37,68.91,0.0,5.0,5.0,6.0,4.0,11.38,8.25,1.33
4,181109,149,70.37,68.91,0.0,5.0,5.0,6.0,4.0,11.38,8.25,1.33
...,...,...,...,...,...,...,...,...,...,...,...,...
59943,181109,234049,70.37,69.28,0.0,27.0,22.0,15.0,5.0,4.50,11.50,4.00
59944,181109,234059,70.37,69.28,0.0,27.0,22.0,15.0,5.0,4.50,11.50,4.00
59945,181109,234109,70.37,69.28,0.0,27.0,22.0,15.0,5.0,4.50,11.50,4.00
59946,181109,234119,70.37,69.28,0.0,27.0,22.0,15.0,5.0,4.50,11.50,4.00


### Plot comparison wrt Altitude

In [14]:
# %load_ext autoreload
# %autoreload 2
# from numpy import array
# import sys
# # sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
# # from SingleSatelliteFlythrough import SingleModelFlythrough
# sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
# from SatelliteFlythrough import ModelFlythrough
# from pymsis import msis


# variables =['lon','lat' ,'ut','ilev','f107d','f107a','Kp', 'O2', 'O1', 'HE', 'DEN', 'TN', 'Z', 'ZG', 'ZGMID']
# tiegcm_file = '/data/data_geodyn/atmos_models_data/tiegcm/2018/Nov2018_ccmcRoR/' +'s%03d.nc' % 25  
# tiegcm = read_tiegcm(tiegcm_file, variables)
# tiegcm  = tiegcm[0]


# rho_tiegcm = []
# rho_kamodo = []
# rho_msise2 = []

# alts_range = np.arange(400,520, 5)


# # for i,val in enumerate(msisinputs['HHMMSS'][0:15]) :    
# for i,val in enumerate(alts_range) :    

#     time =msisinputs['Date'][0]
#     lat = float(msisinputs['Lat'][0])
#     lon = float(msisinputs['Lon'][0])
#     alt = val#float(msisinputs['Height_kilometers'][i])
    
#     print(f'|——————————————————————————————————————————————————————————————————————————')
#     print(f'|  Time: {time}')
#     print(f'|  Lat:  {lat}')
#     print(f'|  Lon:  {lon}')
#     print(f'|  Alt:  {alt}')
# #     maxalt = np.max([tiegcm['ZG'][indexut,:-2, indexlat0, indexlon0 ].max(), tiegcm['ZG'][indexut,:-2, indexlat1, indexlon1 ].max()]) 
# #     print('                        Top boundary TIEGCM file: ',maxalt/100000)
#     print(f' ')

    
#     ##############################################################################
#     ##### CALL TIEGCM Manually---------------------------------------------------
#     ##############################################################################
# #     time_hour = datetime.strptime(sat_time1+sat_time2, '%y%m%d%H%M%S').hour
#     den_TIEGCM_sat_alt = interpolate_tiegcm(tiegcm, lon, lat, time.hour, alt*100000, 'DEN')*1e3
#     rho_tiegcm.append(den_TIEGCM_sat_alt)

#     print('    TIEGCM Manual   : ',den_TIEGCM_sat_alt ,' kg/m^3')  
# #     print('----------------------------------------------------------------------')

#     ##############################################################################
#     ##### CALL KAMODO ------------------------------------------------------------
#     ##############################################################################
#     #### Kamodo static inputs:
#     model          = 'TIEGCM'
#     file_dir       = '/data/data_geodyn/atmos_models_data/tiegcm/2018/Nov2018_ccmcRoR/'
#     variable_list  = ['rho','psi_O2', 'psi_O',  'psi_He', 'T_n']
#     coord_type     = 'SPH'
#     coord_grid     = 'sph'
#     high_res       = 1.
#     verbose        = False  
#     csv_output     = ''
#     plot_output    = ''        

    
    
#     results = ModelFlythrough(model,
#                               file_dir,
#                               variable_list,
#                               [msisinputs['sattime_utctimestamp'][i]],
#                               [lon],
#                               [lat],
#                               [alt],
#                               coord_type,
#                               coord_grid,
#                               high_res=20.,
#                               verbose=False,
#                               csv_output='',
#                               plot_output='')
#     rho_kamodo.append(results['rho'][0]*1e3)
# #     print()
#     print('    Kamodo w extrap : ',results['rho'][0]*1e3, ' kg/m^3')


#     ##############################################################################
#     ##### CALL MSIS2 ---------------------------------------------------
#     ##############################################################################
#     dates = time
#     f107 = msisinput_gpi['flux_daily'][i]
#     f107a = msisinput_gpi['flux_avg'][i]
#     aps = [[msisinput_gpi['Ap1'][i],
#             msisinput_gpi['Ap2'][i],
#             msisinput_gpi['Ap3'][i],
#             msisinput_gpi['Ap4'][i],
#             msisinput_gpi['Ap5'][i],
#             msisinput_gpi['Ap6'][i],
#             msisinput_gpi['Ap7'][i] ]]

#     output2 = msis.run(dates, lon, lat, alt, f107, f107a, aps, version = 2)
#     print('    MSIS2           : ',output2[0][0][0][0][0] , '          kg/m^3')
#     rho_msise2.append(output2[0][0][0][0][0])
    
    
    
    
# %load_ext autoreload
# %autoreload 2
# import plotly.graph_objects as go
# from plotly.offline import plot, iplot
# from plotly.subplots import make_subplots
# import plotly.express as px
# import pandas as pd

# import numpy as np

# config = dict({
#                 'displayModeBar': True,
#                 'responsive': False,
#                 'staticPlot': False,
#                 'displaylogo': False,
#                 'showTips': False,
#                 })

# # Simplify Plotting Schemes:
# col1 = px.colors.qualitative.Plotly[0]
# col2 = px.colors.qualitative.Plotly[1]
# col3 = px.colors.qualitative.Plotly[2]
# col4 = px.colors.qualitative.Plotly[3]
# col5 = px.colors.qualitative.Plotly[4]
# col6 = px.colors.qualitative.Plotly[5]




# fig = make_subplots(
#     rows=1, cols=1,
#     subplot_titles=(['Compare Model Densities wrt to Altitude (lon, lat, time are constant)']),
#     vertical_spacing = 0.15)

# fig.add_trace(go.Scattergl(  x=alts_range,
#                              y=rho_tiegcm,
#                              name= 'manually index tiegcm (no extrapolation done)',
#                              mode='markers',
#                              opacity=1,
#                              marker=dict(
# #                                 color=col, 
# #                                 size=m_size,
#                                         ),
#                              showlegend=True,
#                           ),
#                               secondary_y=False,
#                                row=1, col=1,
#                           )
# fig.add_trace(go.Scattergl(  x=alts_range,
#                              y=rho_msise2,
#                              name= 'msis2',
#                              mode='markers',
#                              opacity=1,
#                              marker=dict(
# #                                 color=col, 
# #                                 size=m_size,
#                                         ),
#                              showlegend=True,
#                           ),
#                               secondary_y=False,
#                                row=1, col=1,
#                           )
# fig.add_trace(go.Scattergl(  x=alts_range,
#                              y=rho_kamodo,
#                              name= 'kamodo (includes extrap)',
#                              mode='markers',
#                              opacity=1,
#                              marker=dict(
# #                                 color=col, 
# #                                 size=m_size,
#                                         ),
#                              showlegend=True,
#                           ),
#                               secondary_y=False,
#                                row=1, col=1,
#                           )
# fig.update_yaxes( title="rho (kg/m**3)", type='log', exponentformat= 'power',row=1, col=1)
# fig.update_xaxes( title="Altitude", row=1, col=1)

# fig.show(config=config)


# # fig.write_image(plots_dir+ 'Density_along_orbit.jpg', format='jpg')


### Plot comparisons thru time

In [15]:
# %load_ext autoreload
# %autoreload 2
# from numpy import array
# import sys
# # sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
# # from SingleSatelliteFlythrough import SingleModelFlythrough
# sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
# from SatelliteFlythrough import ModelFlythrough
# from pymsis import msis


# variables =['lon','lat' ,'ut','ilev','f107d','f107a','Kp', 'O2', 'O1', 'HE', 'DEN', 'TN', 'Z', 'ZG', 'ZGMID']
# tiegcm_file = '/data/data_geodyn/atmos_models_data/tiegcm/2018/Nov2018_ccmcRoR/' +'s%03d.nc' % 25  
# tiegcm = read_tiegcm(tiegcm_file, variables)
# tiegcm  = tiegcm[0]


# rho_tiegcm = []
# rho_kamodo = []
# rho_msise2 = []

# # alts_range = np.arange(400,520, 5)
# dates_plot = []

# # for i,val in enumerate(msisinputs['HHMMSS'][0:15]) :    
# for i,val in enumerate(msisinputs['Date'][:15]) :    

#     time =msisinputs['Date'][i]
#     lat = float(msisinputs['Lat'][i])
#     lon = float(msisinputs['Lon'][i])
#     alt = float(msisinputs['Height_kilometers'][i])
#     dates_plot.append(time)
    
    
#     print(f'|——————————————————————————————————————————————————————————————————————————')
#     print(f'|  Time: {time}')
#     print(f'|  Lat:  {lat}')
#     print(f'|  Lon:  {lon}')
#     print(f'|  Alt:  {alt}')
# #     maxalt = np.max([tiegcm['ZG'][indexut,:-2, indexlat0, indexlon0 ].max(), tiegcm['ZG'][indexut,:-2, indexlat1, indexlon1 ].max()]) 
# #     print('                        Top boundary TIEGCM file: ',maxalt/100000)
#     print(f' ')

    
#     ##############################################################################
#     ##### CALL TIEGCM Manually---------------------------------------------------
#     ##############################################################################
# #     time_hour = datetime.strptime(sat_time1+sat_time2, '%y%m%d%H%M%S').hour
#     den_TIEGCM_sat_alt = interpolate_tiegcm(tiegcm, lon, lat, time.hour, alt*100000, 'DEN')*1e3
#     rho_tiegcm.append(den_TIEGCM_sat_alt)

#     print('    TIEGCM Manual   : ',den_TIEGCM_sat_alt ,' kg/m^3')  
# #     print('----------------------------------------------------------------------')

#     ##############################################################################
#     ##### CALL KAMODO ------------------------------------------------------------
#     ##############################################################################
#     #### Kamodo static inputs:
#     model          = 'TIEGCM'
#     file_dir       = '/data/data_geodyn/atmos_models_data/tiegcm/2018/Nov2018_ccmcRoR/'
#     variable_list  = ['rho','psi_O2', 'psi_O',  'psi_He', 'T_n']
#     coord_type     = 'SPH'
#     coord_grid     = 'sph'
#     high_res       = 1.
#     verbose        = False  
#     csv_output     = ''
#     plot_output    = ''        

    
    
#     results = ModelFlythrough(model,
#                               file_dir,
#                               variable_list,
#                               [msisinputs['sattime_utctimestamp'][i]],
#                               [lon],
#                               [lat],
#                               [alt],
#                               coord_type,
#                               coord_grid,
#                               high_res=20.,
#                               verbose=False,
#                               csv_output='',
#                               plot_output='')
#     rho_kamodo.append(results['rho'][0]*1e3)
# #     print()
#     print('    Kamodo w extrap : ',results['rho'][0]*1e3, ' kg/m^3')


#     ##############################################################################
#     ##### CALL MSIS2 ---------------------------------------------------
#     ##############################################################################
#     dates = time
#     f107 = msisinput_gpi['flux_daily'][i]
#     f107a = msisinput_gpi['flux_avg'][i]
#     aps = [[msisinput_gpi['Ap1'][i],
#             msisinput_gpi['Ap2'][i],
#             msisinput_gpi['Ap3'][i],
#             msisinput_gpi['Ap4'][i],
#             msisinput_gpi['Ap5'][i],
#             msisinput_gpi['Ap6'][i],
#             msisinput_gpi['Ap7'][i] ]]

#     output2 = msis.run(dates, lon, lat, alt, f107, f107a, aps, version = 2)
#     print('    MSIS2           : ',output2[0][0][0][0][0] , '          kg/m^3')
#     rho_msise2.append(output2[0][0][0][0][0])
    
    
    
    
# %load_ext autoreload
# %autoreload 2
# import plotly.graph_objects as go
# from plotly.offline import plot, iplot
# from plotly.subplots import make_subplots
# import plotly.express as px
# import pandas as pd

# import numpy as np

# config = dict({
#                 'displayModeBar': True,
#                 'responsive': False,
#                 'staticPlot': False,
#                 'displaylogo': False,
#                 'showTips': False,
#                 })

# # Simplify Plotting Schemes:
# col1 = px.colors.qualitative.Plotly[0]
# col2 = px.colors.qualitative.Plotly[1]
# col3 = px.colors.qualitative.Plotly[2]
# col4 = px.colors.qualitative.Plotly[3]
# col5 = px.colors.qualitative.Plotly[4]
# col6 = px.colors.qualitative.Plotly[5]




# fig = make_subplots(
#     rows=1, cols=1,
#     subplot_titles=(['Comparison of Model Densities thru indexed Orbit']),
#     vertical_spacing = 0.15)

# fig.add_trace(go.Scattergl(  x=dates_plot,
#                              y=rho_tiegcm,
#                              name= 'tiegcm',
#                              mode='markers',
#                              opacity=1,
#                              marker=dict(
# #                                 color=col, 
# #                                 size=m_size,
#                                         ),
#                              showlegend=True,
#                           ),
#                               secondary_y=False,
#                                row=1, col=1,
#                           )
# fig.add_trace(go.Scattergl(  x=dates_plot,
#                              y=rho_msise2,
#                              name= 'msis2',
#                              mode='markers',
#                              opacity=1,
#                              marker=dict(
# #                                 color=col, 
# #                                 size=m_size,
#                                         ),
#                              showlegend=True,
#                           ),
#                               secondary_y=False,
#                                row=1, col=1,
#                           )
# fig.add_trace(go.Scattergl(  x=dates_plot,
#                              y=rho_kamodo,
#                              name= 'kamodo',
#                              mode='markers',
#                              opacity=1,
#                              marker=dict(
# #                                 color=col, 
# #                                 size=m_size,
#                                         ),
#                              showlegend=True,
#                           ),
#                               secondary_y=False,
#                                row=1, col=1,
#                           )
# fig.update_yaxes( title="rho (kg/m**3)", type='log', exponentformat= 'power',row=1, col=1)
# fig.update_xaxes( title="Dates", row=1, col=1)

# fig.show(config=config)




### NOTE ON THE ABOVE PLOT, It is the manual index of TIEGCM does not include a top boundary extrapolation so its higher density is a result of this.. 

# Final test, run through as Orbit Cloud file outside of GEODYN and without the cube cell


In [16]:
### some external prep

DEN_csv= msisinputs
# DEN_csv

In [17]:
import sys
sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/flythrough/')
from SatelliteFlythrough import ModelFlythrough
from pymsis import msis

import time
start = time.time()


date_list = []
unixtimes_list = []
lons_list = []
lats_list = []
alts_list = []

count=0
rho_msise2 = []

#### Open the file
#### We will loop thru the DEN CSV and if the file already contains the the date, don't overwrite.
### NOTE: The below will search thru the file to see if the date is already in there but it does 
####      not account for the fact that the data must be written in blocks of 9 points of the cube.  
#####     This is a limitation and should be addressed eventually

delta_deg = 2    # degrees
delta_m = 1000.*1e-3 # meters to kilometers

vals  = np.arange(DEN_csv.index[0],DEN_csv.index[-1]+1)
df = DEN_csv.set_index('Date',drop=False ) 
df['i_vals'] = vals
index_date = df.loc[df.index.max()]['i_vals'].min()

print('Last Date:', DEN_csv['Date'][index_date])
for it,val in enumerate(DEN_csv['Date'][:index_date]):
#     print(val)

    
    date_index = DEN_csv['YYMMDD'][it] + DEN_csv['HHMMSS'][it]
    unix_time  = DEN_csv['sattime_utctimestamp'][it]

    count+=1

    ### Get the coordinates along the orbit:
    lon = float(DEN_csv['Lon'][it])
    lat = float(DEN_csv['Lat'][it])
    alt = float(DEN_csv['Height_kilometers'][it])
    center_coord = [lon, lat, alt]

    lon_plus_delta = lon + delta_deg
    lon_mins_delta = lon - delta_deg
    lat_plus_delta = lat + delta_deg
    lat_mins_delta = lat - delta_deg

    ### WRAP THE LONS AROUND -180 to 180
    if lon_plus_delta < -180:
        lon_plus_delta = np.mod(lon_plus_delta, 180)
    elif lon_plus_delta > 180:
        lon_plus_delta = np.mod(lon_plus_delta, -180)
    else:        
        lon_plus_delta = lon_plus_delta

    if lon_mins_delta < -180:
        lon_mins_delta = np.mod(lon_mins_delta, 180)
    elif lon_mins_delta > 180:
        lon_mins_delta = np.mod(lon_mins_delta, -180)
    else:
        lon_mins_delta = lon_mins_delta
      
        
    if lat_plus_delta < -90:
        lat_plus_delta = np.mod(lat_plus_delta, 90)
    elif lat_plus_delta > 90:
        lat_plus_delta = np.mod(lat_plus_delta, -90)
    else:
        lat_plus_delta = lat_plus_delta

    if lat_mins_delta < -90:
        lat_mins_delta = np.mod(lat_mins_delta, 90)
    elif lat_mins_delta > 90:
        lat_mins_delta = np.mod(lat_mins_delta, -90)
    else:
        lat_mins_delta = lat_mins_delta

        
        
        
    ### Find the coordinates of the cube surround the orbit point:
    A = [lon_plus_delta, lat_plus_delta, alt+delta_m]  # top,    front, left
    B = [lon_plus_delta, lat_mins_delta, alt+delta_m]  # top,    back,  Left
    C = [lon_mins_delta, lat_plus_delta, alt+delta_m]  # top,    front, right
    D = [lon_mins_delta, lat_mins_delta, alt+delta_m]  # top,    back,  right
    E = [lon_plus_delta, lat_plus_delta, alt-delta_m]  # bottom, front, left
    F = [lon_plus_delta, lat_mins_delta, alt-delta_m]  # bottom, back,  left
    G = [lon_mins_delta, lat_plus_delta, alt-delta_m]  # bottom, front, right
    H = [lon_mins_delta, lat_mins_delta, alt-delta_m]  # bottom, back,  right


    ### Store the cube's coordinates in the dictionary index
    cube_corners_and_center = []
    cube_corners_and_center.append(center_coord)
    cube_corners_and_center.append(A)
    cube_corners_and_center.append(B)
    cube_corners_and_center.append(C)
    cube_corners_and_center.append(D)
    cube_corners_and_center.append(E)
    cube_corners_and_center.append(F)
    cube_corners_and_center.append(G)
    cube_corners_and_center.append(H)




    #### Extract the coordinates from each list to plug into Kamodo with vectorization
    lons_in = [item[0] for item in cube_corners_and_center]
    lats_in = [item[1] for item in cube_corners_and_center]
    alts_in = [item[2] for item in cube_corners_and_center]

    ## Gather inputs for Kamodo
    sat_time       = unix_time*np.ones(np.size(alts_in))
    dates          = [date_index]*np.size(alts_in)
    c1             = lons_in
    c2             = lats_in
    c3             = alts_in


    unixtimes_list.extend(sat_time)
    date_list.extend(dates)
    lons_list.extend(c1)
    lats_list.extend(c2)
    alts_list.extend(c3)

    
#     dates = DEN_csv['Date'][it]
#     f107 = msisinput_gpi['flux_daily'][it]
#     f107a = msisinput_gpi['flux_avg'][it]
#     aps = [[msisinput_gpi['Ap1'][it],
#             msisinput_gpi['Ap2'][it],
#             msisinput_gpi['Ap3'][it],
#             msisinput_gpi['Ap4'][it],
#             msisinput_gpi['Ap5'][it],
#             msisinput_gpi['Ap6'][it],
#             msisinput_gpi['Ap7'][it] ]]
#     output2 = msis.run(dates, lon, lat, alt, f107, f107a, aps, version = 2)
# #     print('    MSIS2           : ',output2[0][0][0][0][0] , '          kg/m^3')
#     rho_msise2.append(output2[0][0][0][0][0])
    

    
#### Import Coordinates to Kamodo
##
#### Kamodo static inputs:
model          = 'TIEGCM'
file_dir       =  '/data/data_geodyn/atmos_models_data/tiegcm/2018/Nov2018_ccmcRoR/'
variable_list  = ['rho','psi_O2', 'psi_O',  'psi_He', 'T_n']
coord_type     = 'SPH'
coord_grid     = 'sph'
high_res       = 1.
verbose        = False  
csv_output     = '' 
plot_output    = ''        


print('Running thru Kamodo')
# results = ModelFlythrough(model, file_dir, variable_list, unixtimes_list, lons_list, lats_list, alts_list,
#                           coord_type, coord_grid, high_res=20., verbose=False,csv_output='', plot_output='')


#         print(results)
# with open(self.orbitcloud_csv_file, 'r+') as file:
#     for ii, valrho in enumerate(results['rho']):
#         file.write(f"{datetime.strftime(datetime.fromtimestamp(results['utc_time'][ii]), '%y%m%d%H%M%S')}   {results['c1'][ii]:8.4f}   {results['c2'][ii]:8.4f}   {results['c3'][ii]:8.4f}   {valrho:15.8e} \n")

end = time.time()
elapsed = end - start
print('')
print('')
print('')
print('')
print('Run Time:', elapsed,         'seconds' )
print('Run Time:', elapsed/60,    'minutes' )
print('Run Time:', elapsed/60/60, 'hours'   )


Last Date: 2018-11-09 23:41:29
Running thru Kamodo




Run Time: 0.636052131652832 seconds
Run Time: 0.010600868860880535 minutes
Run Time: 0.00017668114768134224 hours


In [33]:
# for i,val in enumerate(lons_list):
#     print(f'|  Time: {date_list[i]},  Lon:  {lons_list[i]},  Lat:  {lats_list[i]},  Alt:  {alts_list[i]}')

# # A = pd.DataFrame.from_dict(results)

# # A

In [19]:
# %load_ext autoreload
# %autoreload 2
# import plotly.graph_objects as go
# from plotly.offline import plot, iplot
# from plotly.subplots import make_subplots
# import plotly.express as px
# import pandas as pd

# import numpy as np

# config = dict({
#                 'displayModeBar': True,
#                 'responsive': False,
#                 'staticPlot': False,
#                 'displaylogo': False,
#                 'showTips': False,
#                 })

# # Simplify Plotting Schemes:
# col1 = px.colors.qualitative.Plotly[0]
# col2 = px.colors.qualitative.Plotly[1]
# col3 = px.colors.qualitative.Plotly[2]
# col4 = px.colors.qualitative.Plotly[3]
# col5 = px.colors.qualitative.Plotly[4]
# col6 = px.colors.qualitative.Plotly[5]




# fig = make_subplots(
#     rows=1, cols=1,
#     subplot_titles=(['Comparison of Model Densities']),
#     vertical_spacing = 0.15)
# fig.add_trace(go.Scattergl(  x=DEN_csv['Date'],
#                              y=rho_msise2,
#                              name= 'MSIS2',
#                              mode='markers',
#                              opacity=1,
#                              marker=dict(
# #                                 color=col, 
# #                                 size=m_size,
#                                         ),
#                              showlegend=True,
#                           ),
#                               secondary_y=False,
#                                row=1, col=1,
#                           )

# fig.add_trace(go.Scattergl(  x=DEN_csv['Date'],
#                              y=results['rho']*1e3,
#                              name= 'Kamodo-TIEGCM',
#                              mode='markers',
#                              opacity=1,
#                              marker=dict(
# #                                 color=col, 
# #                                 size=m_size,
#                                         ),
#                              showlegend=True,
#                           ),
#                               secondary_y=False,
#                                row=1, col=1,
#                           )

# fig.update_yaxes( title="rho (kg/m**3)", type='log', exponentformat= 'power',row=1, col=1)
# fig.update_xaxes( title="Dates", row=1, col=1)

# fig.show(config=config)


In [20]:
# def orb_avg(den_df, arc):
    
    
#     #### Find the index for the correct date
#     vals  = np.arange(den_df[arc].index[0],den_df[arc].index[-1]+1)
#     df = den_df[arc].set_index('Date',drop=False ) 
#     df['i_vals'] = vals
#     index_date = df.loc[df.index.max()]['i_vals'].min()
    
# #     print('index_date', index_date)
#     lat = np.asarray(den_df[arc]['Lat'][:index_date])
#     time_pd = pd.to_datetime(den_df[arc]['Date'][:index_date])
#     i = np.nonzero( lat[1:]*lat[0:-1]  <  np.logical_and(0 , lat[1:] > lat[0:-1] )  )
#     i = i[0]

#     d_avg = np.zeros(np.size(i))
#     height_avg = np.zeros(np.size(i))
    
# #     print('time_pd',time_pd)

#     time_avg = []
#     d_avg_rolling = []
    
#     roll_avg_count = 0
#     for j in range(np.size(i)-1):
#         d_avg[j]      = np.mean(den_df[arc]['rho (kg/m**3)'  ][i[j] : i[j+1]-1  ]  )
#         height_avg[j] = np.mean(den_df[arc]['Height (meters)'][i[j] : i[j+1]-1  ]  )
# #         mean_time      = np.mean(time_pd[   i[j] : i[j+1]-1  ])
#         t1 = pd.to_datetime(time_pd[ i[j]    ])
#         t2 = pd.to_datetime(time_pd[ i[j+1]-1])
#         datemiddle = pd.Timestamp(t1) + (pd.Timestamp(t2) - pd.Timestamp(t1)) / 2

#         time_avg.append(datemiddle)

#         if roll_avg_count ==1:
#             d_avg_rolling.append(np.mean([ d_avg[j],  d_avg[j-1]]))
#             roll_avg_count =0
            
#         roll_avg_count+=1 
#     d_avg_rolling.append(np.mean([ d_avg[j],  d_avg[j-1]]))
        
#     return(time_avg, d_avg, d_avg_rolling )
    


# vals  = np.arange(obj_m1.__dict__['Density'][arc].index[0],obj_m1.__dict__['Density'][arc].index[-1]+1)
# df = obj_m1.__dict__['Density'][arc].set_index('Date',drop=False ) 
# df['i_vals'] = vals
# index_date = df.loc[df.index.max()]['i_vals'].min()


# time_avg,d_avg, d_avg_rolling = orb_avg(obj_m1.Density, arc)


# fig.add_trace(go.Scattergl(x=time_avg,
#                          y=d_avg_rolling,
# #                                  y=d_avg,
# #                                 name= ' Arc ' +str(i_arc) ,
#                          mode='markers',
#                          marker=dict(
#                          color=col,
#                          size=4,),
#                          showlegend=False,
#                            ),
#                            row=1, col=1,
#                            )


# fig.update_yaxes(type="log", exponentformat= 'power',row=1, col=1)
# fig.update_xaxes(title_text="Date", row=1, col=1)
# fig.update_yaxes(title_text="kg/m^3", row=1, col=1)
# fig.update_layout(legend= {'itemsizing': 'constant'})




In [49]:
# lon0     = -175
# loncheck =  -178
# lon1     =  176

# if lon0 < 0:
#     lon0= np.mod(lon0+360,360)
# if lon1 < 0:
#     lon1= np.mod(lon1+360,360)
# if loncheck < 0:
#     loncheck= np.mod(loncheck+360,360)


print(f" {lon0}     {loncheck}        {lon1}")

 -175     -178        176


In [ ]:
np.abs(lon0-loncheck)